In [ ]:
!pip install transformers
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

In [ ]:
import os 
import torch 
import pickle 
import pandas as pd 
import seaborn as sns
from tqdm import tqdm
from time import time
import seaborn as sn
from matplotlib import pyplot as plt 
from torch.nn import LSTM
import torch.nn.functional as F
import numpy as np 
import datetime
import random
from collections import Counter
from shutil import copytree
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, LongformerTokenizer, \
    LongformerForSequenceClassification, BertForSequenceClassification,AdamW,\
    get_cosine_schedule_with_warmup, BertTokenizerFast, BertModel
from sklearn.metrics import balanced_accuracy_score, f1_score
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as Opt
#torch.set_default_tensor_type(torch.HalfTensor)
#torch.set_default_tensor_type(torch.cuda.HalfTensor)
import mlflow
from pyngrok import ngrok

In [ ]:
!cp '/content/as_tokenized_script_chunks.pickle'  '/content/drive/MyDrive/'

In [ ]:
!cp '/content/as_scripts.pickle'  '/content/drive/MyDrive/'

In [ ]:
config = { 
        'paths': { 
            #'nominated_movies': '/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/Screenplay awards data/all_awards_movies.pickle',
            'scripts' : '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Scripts',
            #'movies_matching_scores' : '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie characters/Matching evaluation and statistics/movies_mean_matching_scores.xlsx',
            #'logs_dir':'/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/logs',
            'ckpt_dir': '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/ckpts',
            #'ckpt_dir': '/content/ckpts',
            #'ckpt_to_load': '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/ckpts/script_genre_classification_10_scenes_average_checkpoint_985',
            'ckpt_to_load': '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/ckpts/script_genre_classification_10_scenes_average_checkpoint_398',
            'script_annotations': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Script annotations by BERT/row_classification',
            #
            #'model_annotations': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Script annotations by BERT',
            #'manual_annotations': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Script manual annotations',
             'mlruns': '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/mlruns',
             'script_lemmas':'/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Scripts_lemmas',
             'task_to_labels': '/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/Script texts/script_task_to_labels_dicts.pickle'
        },
        'train': {
                'optim' : {
                    'AdamW':{
                        'lr':1e-5,
                        'eps': 1e-8,
                        'weight_decay':0.0001
                    }
                },
                #'grad_accum_iters': 2, 
                'embedding_size': 768, 
                'model': BertForSequenceClassification, # from [BertForSequenceClassification, LongformerForSequenceClassification]
                'tokenizer':BertTokenizerFast, # from [BertTokenizer, LongformerTokenizer]
                'pretrained_model_type': 'bert-base-cased', # from ['bert-base-cased', 'allenai/longformer-base-4096']
                #'max_script_length' : 65540, #  131072, 65540, 32770
                'max_seq_length' : 512,
                'num_classes' : 2, # [2, 13]
                'max_scene_number' : 100, 
                'nrof_steps_for_shed' : 800,
                'nrof_epochs' : 2,  
                'tr_batch_size' : 4,
                'tst_batch_size' : 4,
                'exp_name':'script_awards_classification_100_scenes_average', 
                'task_name':'script_awards',
                'classes_names': {'genre':['Comedy', 'Short', 'Thriller', 'Documentary', 'Horror', 'Biography', 'Mystery', 
                                           'Animation', 'Crime', 'Adventure', 'Action', 'Drama', 'Fantasy'],
                                  'script_awards':['Not nominated', 'Nominated']
                                  },
                'use_intermediate_weights': False,
                'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                #'heading_to_class_map': {'scene_heading':0, 'text':1, 'speaker_heading':2, 'dialog':3},
                #'class_to_heading_map': {0:'scene_heading', 1:'text', 2:'speaker_heading', 3:'dialog'},
                'load_model':False
                }
        }

In [ ]:
with open('/content/drive/MyDrive/labels.pickle', 'rb') as f:
    labels = pickle.load(f)


In [ ]:
print(labels)

In [ ]:
genre_classes_names=['News', 'Western', 'Musical', 'Thriller', 'Family', 'Animation',
                                                         'Action', 'History', 'Biography', 'Crime', 'Mystery', 'War', 'Sci-Fi', 
                                                        'Fantasy', 'Music', 'Film-Noir', 'Horror', 'Talk-Show', 'Adventure', 'Romance']

In [ ]:
with open(config['paths']['task_to_labels'], 'rb') as f:
    task_to_labels_dicts = pickle.load(f)

In [ ]:
with open(config['paths']['task_to_labels'], 'wb') as f:
    pickle.dump(task_to_labels_dicts, f)

In [ ]:
print(task_to_labels_dicts.keys())

In [ ]:
print(task_to_labels_dicts['genre'])

In [ ]:
print(list(set(task_to_labels_dicts['genre'].values())))

In [ ]:
print(task_to_labels_dicts['genre'])

In [ ]:
print(-1 in task_to_labels_dicts['genre'].values())

In [ ]:
task_to_labels_dicts['genre'] = dict([(str(k).rjust(7,'0'), v) for k, v in task_to_labels_dicts['genre'].items()])

In [ ]:
new_dict = {}
for key, value in task_to_labels_dicts['genre'].items():
    new_dict[str(key).rjust('0')] = value

In [ ]:
print()

In [ ]:
print(task_to_labels_dicts['script_awards'])

In [ ]:
print(task_to_labels_dicts['meta_scores'])

In [ ]:
print(task_to_labels_dicts['meta_scores'])

In [ ]:
task_to_labels_dicts['meta_scores'] = dict([(str(k).rjust(7,'0'), v) for k, v in task_to_labels_dicts['meta_scores'].items()])

In [ ]:
print(task_to_labels_dicts['year'])

In [ ]:
print(task_to_labels_dicts['year'])

In [ ]:
task_to_labels_dicts['year'] = dict([(str(k).rjust(7,'0'), v) for k, v in task_to_labels_dicts['year'].items()])

In [ ]:
with open('/content/labels.pickle', 'rb') as f:
    labels = pickle.load(f)
print(len(labels))

In [ ]:
from collections import Counter

In [ ]:
counter = Counter(task_to_labels_dicts['genre'].values())
print(counter)
print(counter['Comedy'])

In [ ]:
print(len(set(task_to_labels_dicts['genre'].values())))

In [ ]:
genre_list = list(set(task_to_labels_dicts['genre'].values()))
genre_dict = dict(zip(genre_list, range(len(genre_list))))
print(genre_dict)

## Some stuff:

In [ ]:
path_to_awards = '/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/Screenplay awards data'
events = ('Academy Awards adapted screenplay', 'Academy Awards original screenplay',
          'BAFTA nominations', 'Golden Globe Award for Best Screenplay',
          'Writers Guild Awards Winners & Nominees 2020-2013')
file_names  =('academy_awards_adapted_screenplay_movies', 'academy_awards_original_screenplay_movies',
              'bafta_screenplay_movies', 'golden_globe_best_screenplay_movies',
              'writers_guild_awards_movies')

movies_awards = {}

for event, file_name in zip(events,file_names):
    with open(os.path.join(path_to_awards, file_name + '.pickle'), 'rb') as f:
        movies = pickle.load(f)
    for movie in movies:
        if movie in movies_awards:
            movies_awards[movie].append(event)
        else:
            movies_awards[movie]=[event]

awards_df = {'movie':[], 'Academy Awards adapted screenplay':[],
             'Academy Awards original screenplay':[], 'BAFTA nominations':[],
             'Golden Globe Award for Best Screenplay':[], 
             'Writers Guild Awards Winners & Nominees 2020-2013':[]}
for movie in movies_awards:
    awards_df['movie'].append(movie.split('.')[0])
    for key in events:
        if key in movies_awards[movie]:
            awards_df[key].append('+')
        else:
            awards_df[key].append('-')


awards_df = pd.DataFrame(awards_df)
print(awards_df.head())
    

## Dataset preprocessing:

### Additional:

In [ ]:
nominated_movies_imdb_ids, not_nominated_movies = choose_imdb_ids_for_training(config)

In [ ]:
scripts_awards_df = {'scripts':[], 'label':[]}

for file_name in tqdm(os.listdir(config['paths']['scripts'])):
    imdb_id = file_name.split('_')[1].replace('.txt','')
    text = read_script(os.path.join(config['paths']['scripts'], file_name))
    scripts_awards_df['scripts'].append(text)
    scripts_awards_df['label'].append(int(imdb_id in nominated_movies_imdb_ids))
    


In [ ]:
scripts_awards_df = pd.DataFrame(scripts_awards_df)
scripts_awards_df.head()

In [ ]:
with open('scripts_awards_df.pickle', 'wb') as f:
    pickle.dump(scripts_awards_df, f)

In [ ]:
!cp 'scripts_awards_df.pickle' '/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/'

### Main:

In [ ]:
tokenizer = config['train']['tokenizer'].from_pretrained(config['train']['pretrained_model_type'], 
                                          do_lower_case=False)

In [ ]:
'''
def choose_imdb_ids_for_training(config, nrof_not_nominated = 5000):
    with open(config['paths']['nominated_movies'], 'rb') as f:
        nominated_movies = pickle.load(f)
    nominated_movies_imdb_ids = [x.split('_')[1].split('.')[0] for x in  nominated_movies]
    mean_matching_scores_df = pd.read_excel(config['paths']['movies_matching_scores'])
    imdb_ids = mean_matching_scores_df[mean_matching_scores_df.iou_values_mean<1.]['imdb_id'].tolist()
    imdb_ids = [str(x).rjust(7,'0') for x in imdb_ids]
    not_nominated_movies = [x for x in imdb_ids if not x in nominated_movies_imdb_ids]

    return nominated_movies_imdb_ids, not_nominated_movies[:nrof_not_nominated]
'''
def read_script(path):
    try:
        with open(path, 'r') as f:
            text = f.read()
    except:
        with open(path, 'r', encoding='latin-1') as f:
            text = f.read()

    return text 

def show_histogram(x_data, y_data, values_to_show=None, figsize=(40, 10), x_label='x', y_label='y',
                   set_rotation=False, rotation_angle=45, title='title', file_path='name.png', dpi=100, to_save=False, to_show=True):

    fig = plt.figure(figsize=figsize)
    h = sns.barplot(x=x_data, y=y_data, palette="Blues_d")
    if values_to_show:
        for i,y in enumerate(y_data):
            h.text(i, y, str(y)+'\n('+str(values_to_show[i]) + ')', color='black', ha='center')
    if set_rotation:
        h.set_xticklabels(h.get_xticklabels(), rotation=rotation_angle)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if to_show:
        plt.show()

    if to_save:
        fig.savefig(file_path, dpi=dpi, bbox_inches='tight')
        
def remove_labels(row):
    if row.startswith('text:'):
        row = row[6:]
    elif row.startswith('dialog:'):
        row = row[8:]
    elif row.startswith('speaker_heading:'):
        row = row[17:]
    elif row.startswith('scene_heading:'):
        row = row[15:]
    return row.strip()

'''
def plot_conf_matr(trues, predicts, title='tr_'):
    results = np.zeros((2,2))
    for t, p in zip(trues, predicts):
            results [t][p]+=1
    df_cm = pd.DataFrame(results.astype(np.int), index = ['not nominated', 'nominated'],
                  columns = ['not nominated', 'nominated'])
    plt.figure(figsize = (7,7))    
    sn.heatmap(df_cm, annot=True, fmt='d')
    plt.savefig(title+'conf_matrix.png', bbox_inches='tight')
    plt.close()
'''

In [ ]:
class ScriptData:
    def __init__(self, config, tokenizer):
        self.config = config 
        self.tokenizer = tokenizer
        self.imdb_to_labels_dict = self.get_labels_dict()
        self.scripts, self.labels, self.imdb_ids = self._get_scripts_and_labels(to_load=True)
        print('Labels:', self.labels[:10])
        self.scripts_scenes = self.get_script_scenes()
    
    def get_labels_dict(self):
        with open(self.config['paths']['task_to_labels'], 'rb') as f:
            task_to_labels_dict = pickle.load(f)
        if self.config['train']['task_name']=='genre':
            self.genre_dict = dict(zip(self.config['train']['classes_names']['genre'], 
                                       range(len(self.config['train']['classes_names']['genre']))))
            #self.genre_dict = dict(zip(genre_list, range(len(genre_list))))
            print(self.genre_dict)
            #genre_list = list(set(task_to_labels_dict['genre'].values()))
            #genre_list = [x for x in genre_list if not x==-1]
            #self.genre_dict = dict(zip(genre_list, range(len(genre_list))))  
            #self.classes_names = genre_list
        elif self.config['train']['task_name']=='script_awards':
            ...
            #self.classes_names = ['not_nominated', 'nominated']
        return task_to_labels_dict[self.config['train']['task_name']]

    def prepare_label(self, label):
        if self.config['train']['task_name']=='genre':
            return self.genre_dict[label]
        else:
            return label 

    def _get_scripts_and_labels(self, to_load=False):
        label_counter = Counter(self.imdb_to_labels_dict.values())
        #with open(self.config['paths']['labels'], 'rb') as f:
         #   imdb_id_to_label_mapping = pickle.load(f)
        #nominated_movies_imdb_ids, not_nominated_movies_imdb_ids = \
         #               choose_imdb_ids_for_training(self.config)
        if not (to_load and os.path.exists('scripts.pickle') and os.path.exists('labels.pickle')):
            scripts, labels, imdb_ids = [], [], []
            for file_name in tqdm(os.listdir(self.config['paths']['script_annotations'])):
                imdb_id = file_name.split('_')[1]
                if imdb_id in self.imdb_to_labels_dict:
                    if self.imdb_to_labels_dict[imdb_id]!=-1 and label_counter[self.imdb_to_labels_dict[imdb_id]]>10:
                        with open(os.path.join(self.config['paths']['script_annotations'], file_name), 'r') as f:
                            anno_lines = f.readlines()
                        #text = read_script(os.path.join(self.config['paths']['scripts'], file_name))
                        scripts.append(anno_lines)
                        #labels.append(int(imdb_id in nominated_movies_imdb_ids))
                        labels.append(self.prepare_label(self.imdb_to_labels_dict[imdb_id]))
                        imdb_ids.append(imdb_id)
            with open('scripts.pickle', 'wb') as f:
                pickle.dump(scripts, f)
            with open('labels.pickle', 'wb') as f:
                pickle.dump(labels, f)
            with open('imdb_ids.pickle', 'wb') as f:
                pickle.dump(imdb_ids, f)
            print('Data saved')
        else:
            with open('scripts.pickle', 'rb') as f:
                scripts = pickle.load(f)
            with open('labels.pickle', 'rb') as f:
                labels = pickle.load(f)
            with open('imdb_ids.pickle', 'rb') as f:
                imdb_ids = pickle.load(f)
            print('Data loaded')

        
        return scripts, labels, imdb_ids
    '''
    def preprocess_text(self, text):
        text = re.sub(" +|\n+|\r|\t|\0|\x0b|\xa0|\x80|\x93|\x99", ' ', text).strip()
        text = ' '.join(re.findall(r'[a-zA-Z0-9!"#%&\'()+,\-\./:;?\[\]]+', text)).strip()
        text = re.sub(' \.', '.', text)
        return text
    '''
    def make_weights_for_balanced_classes(self, labels):     
        nclasses = self.config['train']['num_classes']
        count = [0] * nclasses

        for label in labels:                                                         
            count[label] += 1                                                     
        weight_per_class = [0.] * nclasses                                     
        N = float(sum(count))                                                   
        for i in range(nclasses):                                                   
            if count[i]!=0:
                weight_per_class[i] = N/float(count[i])                                 
            else:
                weight_per_class[i] = 1000
        weight = [0] * len(labels)                                              
        for idx, label in enumerate(labels):                                          
            weight[idx] = weight_per_class[label]

        print('Weights per class:', weight_per_class)
        return weight, weight_per_class
    '''
    def tokenize_scripts(self, to_load=True):
        if to_load and os.path.exists('tokenized_scripts.pickle'):
            with open('tokenized_scripts.pickle', 'rb') as f:
                tokenized_scripts = pickle.load(f)
        else:
            tokenized_scripts = self.tokenizer(self.scripts)['input_ids']
            tokenized_scripts = [x[1:] for x in tokenized_scripts] # remove cls 
            with open('tokenized_scripts.pickle', 'wb') as f:
                pickle.dump(tokenized_scripts, f)
        return tokenized_scripts
    '''
    def prepare_tokenized_chunks_masks_labels(self, 
                                              tokenized_script_chunks, 
                                              attention_masks,
                                              labels,
                                              imdb_ids):
        tokenized_script_chunks = [torch.tensor(chunk) for chunk in tokenized_script_chunks]
        attention_masks = [torch.tensor(mask) for mask in attention_masks]
        padded_tokenized_script_chunks = pad_sequence(tokenized_script_chunks, 
                                                      padding_value=0, batch_first=True)# for BERT pad = 0 ?
        padded_attention_masks = pad_sequence(attention_masks, 
                                              padding_value=0, batch_first=True)
        labels = torch.LongTensor(labels)
        imdb_ids = [int(x) if x else 0 for x in imdb_ids]
        imdb_ids = torch.LongTensor(imdb_ids)

        return padded_tokenized_script_chunks, padded_attention_masks, labels, imdb_ids

    def get_script_scenes(self):
        scripts_scenes = []

        scene_text = ''
        for i, script_lines in enumerate(self.scripts):
            scripts_scenes.append([])
            for line_num, line in enumerate(script_lines):
                if line.startswith('scene_heading:'):
                    if scene_text:
                        scripts_scenes[-1].append(scene_text)
                    scene_text = remove_labels(line)
                else:
                    scene_text+=remove_labels(line)
            if not scripts_scenes[-1]:
                scripts_scenes.pop(-1)
                self.labels.pop(i)
                self.imdb_ids.pop(i)

        return scripts_scenes

    def make_tokenized_chunks_for_scripts(self, to_load=True):
        if not (to_load and os.path.exists('tokenized_script_chunks.pickle') and os.path.exists('attention_masks.pickle')):
            tokenized_script_chunks, attention_masks = [], []
            text_iterator = self.scripts_scenes #if self.config['train']['scene_as_chunk'] else self.scripts 

            for script_scenes in tqdm(text_iterator):
                if len(script_scenes)>0:
                    tokenized_scenes = self.tokenizer(script_scenes, truncation=True, 
                                                    padding='max_length',
                                                    return_attention_mask=True) #['input_ids']
                    random_indices = sorted(random.sample(range(len(tokenized_scenes['input_ids'])), k =min(len(tokenized_scenes['input_ids']),
                                                                                                            self.config['train']['max_scene_number'])))
                    #print('random indices:', random_indices)
                    #print()
                    tokenized_scenes_ids = tokenized_scenes['input_ids']
                    tokenized_scenes_ids = [x for i, x in enumerate(tokenized_scenes_ids) if i in random_indices]
                    #tokenized_scenes_attention_masks = tokenized_scenes['attention_mask'][:self.config['train']['max_scene_number']]
                    tokenized_scenes_attention_masks = tokenized_scenes['attention_mask']
                    tokenized_scenes_attention_masks = [x for i, x in enumerate(tokenized_scenes_attention_masks) if i in random_indices]
                    tokenized_script_chunks.append(tokenized_scenes_ids)
                    attention_masks.append(tokenized_scenes_attention_masks)
            with open('tokenized_script_chunks.pickle', 'wb') as f:
                pickle.dump(tokenized_script_chunks, f)
            with open('attention_masks.pickle', 'wb') as f:
                pickle.dump(attention_masks, f)
        else:
            with open('tokenized_script_chunks.pickle', 'rb') as f:
                tokenized_script_chunks = pickle.load(f)
            with open('attention_masks.pickle', 'rb') as f:
                attention_masks = pickle.load(f)
                
        return tokenized_script_chunks, attention_masks

        '''
        def make_tokenized_chunks_for_scripts(self, to_load=False):
            tokenized_scripts = self.tokenize_scripts()
            tokenized_script_chunks, attention_masks = [], []
            max_script_l = self.config['train']['max_script_length']
            max_seq_l = self.config['train']['max_seq_length']
            n = int(max_script_l / (max_seq_l//2)) - 1
            
            for t_script in tqdm(tokenized_scripts):    
                t_script = t_script[:max_script_l]
                n = int(len(t_script) / (max_seq_l))
                script_chunks_masks = [(t_script[i*(max_seq_l):i*(max_seq_l) + max_seq_l - 1], [1]*(min(max_seq_l, len(t_script) - i*(max_seq_l - 1)))) for i in range(n)]

                if len(script_chunks_masks[-1][0]) < max_seq_l - 1:
                    script_chunks_masks[-1] = (script_chunks_masks[-1][0] + [1] * (max_seq_l -1 - len(script_chunks_masks[-1][0])),
                                            script_chunks_masks[-1][1] + [0] * (max_seq_l -1 - len(script_chunks_masks[-1][1])))
                
                script_chunks, script_masks = zip(*script_chunks_masks)
                script_chunks = [[101] + x for x in script_chunks]
                script_masks = [[1] + x for x in script_masks]
                tokenized_script_chunks.append(script_chunks)
                attention_masks.append(script_masks)

            return tokenized_scripts, tokenized_script_chunks, attention_masks
        '''
    def get_train_val_split(self):
        tokenized_script_chunks, attention_masks = self.make_tokenized_chunks_for_scripts()
        tr_inputs, val_inputs, tr_masks, val_masks, tr_labels, val_labels, tr_imdb_ids, val_imdb_ids = train_test_split(
            tokenized_script_chunks, attention_masks, self.labels,  self.imdb_ids,
            test_size=0.2, random_state=11, stratify=self.labels)
        self.weights, self.weights_per_class = self.make_weights_for_balanced_classes(tr_labels)
        tst_inputs, val_inputs, tst_masks, val_masks, tst_labels, val_labels, tst_imdb_ids, val_imdb_ids = train_test_split(
            val_inputs, val_masks, val_labels, val_imdb_ids, 
            test_size=0.25, random_state=11)
        classes_info = 'Train set: '+str(len(tr_inputs))+'\nVal set: ' + str(len(val_inputs)) +'\nTest set: '+str(len(tst_inputs))
        with open('dataset_info.txt', 'w') as f:
            f.write(classes_info)
        mlflow.log_artifact('dataset_info.txt')

        tr_inputs, tr_masks, tr_labels, tr_imdb_ids = self.prepare_tokenized_chunks_masks_labels(
            tr_inputs, tr_masks, tr_labels, tr_imdb_ids)
        val_inputs, val_masks, val_labels, val_imdb_ids = self.prepare_tokenized_chunks_masks_labels(
            val_inputs, val_masks, val_labels, val_imdb_ids)
        tst_inputs, tst_masks, tst_labels, tst_imdb_ids = self.prepare_tokenized_chunks_masks_labels(
            tst_inputs, tst_masks, tst_labels, tst_imdb_ids)
        
        return tr_inputs, tr_masks, tr_labels, val_inputs, val_masks, val_labels, tst_inputs, tst_masks, tst_labels, tr_imdb_ids, val_imdb_ids, tst_imdb_ids

In [ ]:
def get_dataloader(input_ids, labels, attention_masks, imdb_ids, 
                   batch_size=1,  phase='train', sampler=None):
        dataset = TensorDataset(input_ids, attention_masks, labels, imdb_ids)
        if phase=='train':
            sampler = sampler if not sampler is None else RandomSampler(dataset)
            dataloader = DataLoader(
                        dataset,  
                        batch_size = batch_size,
                        sampler = sampler,
                        drop_last=True
                    )
        else:
            dataloader = DataLoader(
                        dataset,  
                        batch_size = batch_size,
                        drop_last=True
                    )

        return dataloader 

In [ ]:
def get_data_loaders():
    SD = ScriptData(config, tokenizer)
    #tr_inputs, val_inputs, tst_inputs, tr_labels, val_labels, tst_labels = AD.get_train_val_split()
    tr_inputs, tr_masks, tr_labels, val_inputs, val_masks, val_labels, tst_inputs, tst_masks, tst_labels, tr_imdb_ids, val_imdb_ids, tst_imdb_ids = SD.get_train_val_split()
    #tr_inputs, tr_attention_masks, tr_labels, val_inputs, val_attention_masks, \
    #val_labels, tst_inputs, tst_attention_masks, tst_labels = SD.get_train_val_split()

    #tr_loader = get_dataloader(tr_inputs[[0, 1, 3, 4]], tr_labels[[0, 1, 3, 4]],  tr_attention_masks[[0, 1, 3, 4]], # [0, 1, 3, 4]
    tr_loader = get_dataloader(tr_inputs, tr_labels,  tr_masks, tr_imdb_ids, 
                            batch_size=config['train']['tr_batch_size'],
                            sampler = WeightedRandomSampler(SD.weights, len(SD.weights)))
    val_loader = get_dataloader(val_inputs, val_labels,  val_masks, val_imdb_ids, 
                                batch_size=config['train']['tst_batch_size'])
    tst_loader = get_dataloader(tst_inputs, tst_labels,  tst_masks, tst_imdb_ids,
                                batch_size=config['train']['tst_batch_size'])
    
    return tr_loader, val_loader, tst_loader


### Additional x2:

In [ ]:
with open('/content/labels.pickle', 'rb') as f:
    labels = pickle.load(f)
with open('/content/labels_true.pickle', 'rb') as f:
    labels_true = pickle.load(f)

In [ ]:
print(labels[:20])
print(labels_true[:20])

In [ ]:
print(labels == labels_true)

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

SD = ScriptData(config, tokenizer)

In [ ]:
print(SD.classes_names)

In [ ]:
print(len(SD.classes_names))

In [ ]:
print(len(SD.scripts))

In [ ]:
print(dir(tokenizer))

In [ ]:
print(tokenizer.special_tokens_map)

In [ ]:
tokenized_chunks_for_scripts, attention_masks = SD.make_tokenized_chunks_for_scripts()

In [ ]:
print(len(tokenized_chunks_for_scripts[0][0]))
print(len(tokenized_chunks_for_scripts[0]))
print(len(tokenized_chunks_for_scripts))

In [ ]:
from itertools import chain

In [ ]:
scrit_scenes = list(chain(*tokenized_chunks_for_scripts))

In [ ]:
print(len(scrit_scenes))

In [ ]:
print(len(scrit_scenes[0]))

In [ ]:
scene_lengths = [len(x) for x in scrit_scenes]

In [ ]:
script_scenes_count = [len(x) for x in tokenized_chunks_for_scripts]
print(len(script_scenes_count))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

tr_inputs, tr_masks, tr_labels, val_inputs, val_masks, val_labels, tst_inputs, tst_masks, tst_labels, tr_imdb_ids, val_imdb_ids, tst_imdb_ids = SD.get_train_val_split()

In [ ]:
print(tr_labels[:20])
print(tr_imdb_ids[:20])

In [ ]:
print(len(tr_labels))

In [ ]:
with open(os.path.join('/content/drive/MyDrive/Data for baseline/Awards', 'val_labels.pickle'), 'wb') as f:
    pickle.dump(val_labels, f)

In [ ]:
with open(os.path.join('/content/drive/MyDrive/Data for baseline/Awards', 'tr_imdb_ids.pickle'), 'wb') as f:
    pickle.dump(tr_imdb_ids, f)

In [ ]:
print(tr_labels[:10])
#print(tr_labels[[0, 1, 3, 4]])

In [ ]:
print(tr_inputs.shape)
print(tr_inputs[0].shape)
print(tr_inputs[1].shape)

In [ ]:
imdb_ids_to_labels = {'tr': dict(zip(tr_imdb_ids, tr_labels)), 'val':dict(zip(val_imdb_ids, val_labels)), 'tst':dict(zip(tst_imdb_ids, tst_labels))}

In [ ]:
print(tr_labels[:10])

In [ ]:
print(tr_inputs[0])

In [ ]:
print(tr_inputs[1])

In [ ]:
print(tokenizer.convert_ids_to_tokens(tr_inputs[3][0]))

In [ ]:
print(tokenizer.convert_ids_to_tokens(tr_inputs[3][1]))

In [ ]:
print(tokenizer.convert_ids_to_tokens(tr_inputs[3][2]))

In [ ]:
!cp '/content/tokenized_scripts.pickle' '/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/'

In [ ]:
with open('tokenized_scripts.pickle', 'rb') as f:
    tokenized_scripts = pickle.load(f)


In [ ]:
print(len(tokenized_scripts))

In [ ]:
print(len(tokenized_scripts))

In [ ]:
script_num = 2
print(tokenized_scripts[script_num])
print(tokenized_chunks_for_scripts[script_num])
print(attention_masks[script_num])

In [ ]:
print(len(tokenized_scripts[script_num]))
print(len(tokenized_chunks_for_scripts[script_num]))
for chunk in tokenized_chunks_for_scripts[script_num]:
    print(len(chunk))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

tr_loader, val_loader, tst_loader = get_data_loaders()

In [ ]:
!cp '/content/tokenized_script_chunks.pickle' '/content/drive/MyDrive'

In [ ]:
with open('/content/attention_masks.pickle', 'rb') as f:
    attention_1 = pickle.load(f)

In [ ]:
print(len(attention_1))

In [ ]:
print(attention_1[0])

In [ ]:
with open('/content/drive/MyDrive/attention_masks.pickle', 'rb') as f:
    attention_2 = pickle.load(f)

In [ ]:
print(attention_2[0])

In [ ]:
print(attention_1==attention_2)

In [ ]:
for i, d in enumerate(tr_loader):
    print(d)
    print(d[0].shape)
    print(d[1].shape)
    print(d[2].shape)
    print()
    if i>5:
        break 

In [ ]:
ckpt = torch.load(config['paths']['ckpt_to_load'],
                                       #'script_classification_with_val_checkpoint_val_0_5955'),
                          map_location=config['train']['device'])

In [ ]:
print(ckpt.keys())

In [ ]:
print(ckpt['model'].keys())

In [ ]:
transformer_model = BertModel.from_pretrained(
                config['train']['pretrained_model_type'],
                gradient_checkpointing=True) 

In [ ]:
print(list(transformer_model.named_parameters()))

In [ ]:
print(transformer_model)

### Scripts length stats:

In [ ]:
print(max(scene_lengths))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 

#lengths = [len(x) for x in tokenized_scripts]
lengths = scene_lengths
bins = list([0,100,200,300,400, 512] + list(range(600,3000,200)) + [3000, 31688])
h, bins = np.histogram(lengths, bins=bins)
h_cum = np.cumsum(h)
nrof_movies = np.sum(h)
h_cum_to_show = [str(x)+ ':\n'+str(round(float(x)/nrof_movies, 3)) for x in h_cum]
file_path = os.path.join('/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/scripts_scenes_lengths_distribution.png')
#file_path = 'Scripts_scenes_count_distribution.png'
bins = [str(bins[i])+'-'+str(bins[i+1])for i in range(len(bins)-1)]
show_histogram(bins, h, figsize=(20,15), x_label='Number of tokens', values_to_show= h_cum_to_show, y_label='Number of scenes',
               set_rotation=True, title='Scripts scenes lengths distribution', file_path=file_path,
               dpi=200, to_save=True)

## Model:

In [ ]:
class TransformerQA(nn.Module):
    def __init__(self, model, confg): 
        super(TransformerQA, self).__init__()
        self.transformer_model = model
        print('bert params:')
        self.dropout = nn.Dropout(0.1)
        #print(self.transformer_model.parameters())
        for param in self.transformer_model.parameters():
            param.requires_grad = True 

        #self.cls_ff = torch.nn.Linear(config['train']['embedding_size'], 2)
        self.cls_ff = nn.Linear(config['train']['embedding_size'], config['train']['num_classes'])
        #self.cls_ff = nn.Sequential(
          #  nn.Dropout(0.1),
           # nn.Linear(config['train']['embedding_size'], config['train']['num_classes'])
         #   )
        self.lstm = LSTM(config['train']['embedding_size'],config['train']['embedding_size'])
        '''
        self.cls_ff = nn.Sequential(
            nn.Linear(config['train']['embedding_size'], 3072),
            nn.Linear(3072, config['train']['embedding_size']),
            nn.LayerNorm((768,), eps=1e-12, elementwise_affine=True),
            nn.Dropout(0.1),
            nn.Linear(config['train']['embedding_size'], config['train']['embedding_size']),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(config['train']['embedding_size'], config['train']['num_classes']),
            #nn.Sigmoid()
        )
        '''

    def forward(self, script, input_masks, label):
        '''
        script squeezed (of size (nrof chunks, max seq len))
        '''
        script_vector = []
        bert_output = torch.zeros(config['train']['tr_batch_size'], 
                                  config['train']['max_scene_number'],
                                  #config['train']['max_script_length'] // config['train']['max_seq_length'],
                                  config['train']['embedding_size'],
                                  device=config['train']['device'])
        #for i, (name, param) in enumerate(self.transformer_model.named_parameters()):                
         #   if param.requires_grad and name=='encoder.layer.11.output.dense.weight':
          #      print(name)
           #     print(param.data)
            #    print()
            
        #print('ff weights', self.cls_ff.weight)
        #print('script.shape', script.shape)
        #### ?????!
        #print('transformer_model params', list(self.transformer_model.parameters())[0][0])
        for i in range(script.shape[1]):   
            #outputs = self.dropout(self.transformer_model(script[:, i, :],
                                 #attention_mask = input_masks[:, i, :][1])  
            #print('BERT OUTPUTS')
            #print(outputs)
            #cls_vector = outputs['hidden_states'][0][:,0,:]
            #print('cls shape', cls_vector.shape)
            #bert_output[:, i, :] = cls_vector
            #bert_output[:, i, :] = outputs[1] #self.dropout(
            bert_output[:, i, :] = self.transformer_model(script[:, i, :],
                                 attention_mask = input_masks[:, i, :]).last_hidden_state[:,0,:]
            #script_vector.append(cls_vector)
       
        #stacked = torch.stack(script_vector)
        script_vector = torch.mean(bert_output, 1)
        #output, (_, _) = self.lstm(bert_output.permute(1,0,2))
        #print('lstm output.shape', output.shape)
        
        #last_layer = self.dropout(F.relu(output[-1]))
        #ff_res = self.cls_ff(last_layer)
        #ff_res = self.cls_ff(output.permute(1,0,2))
        

        ff_res = self.cls_ff(self.dropout(F.relu(script_vector)))

        return ff_res

## Train:

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


def plot_conf_matr(trues, predicts, file_path_prefix='tr_', title='Confusion matrix',
                   nrof_classes=2,  classes_names=['not nominated', 'nominated']):
    results = np.zeros((nrof_classes,nrof_classes))
    for t, p in zip(trues, predicts):
        results[t][p]+=1
        
    df_cm = pd.DataFrame(results.astype(np.int), index = classes_names,
                  columns = classes_names)
    plt.figure(figsize = (7,7))    
    ax = sns.heatmap(df_cm, annot=True, fmt='d')
    ax.set(xlabel='predicted', ylabel='actual',title=title)
    plt.savefig(file_path_prefix+'conf_matrix.png', bbox_inches='tight')
    plt.close()
    

In [ ]:
class Train():
    def __init__(self, config):
        self.config = config 
        transformer_model = BertModel.from_pretrained(
                config['train']['pretrained_model_type'],
                gradient_checkpointing=True) 
                #output_attentions = False, 
                #output_hidden_states = True, 
             #   gradient_checkpointing=False
              #  )
        #transformer_model = self.config['train']['model'].from_pretrained(
         #       config['train']['pretrained_model_type'], 
          #      num_labels = self.config['train']['num_classes'],
           #     output_attentions = False, 
            #    output_hidden_states = True, 
             #   gradient_checkpointing=False
              #  )
        self.model = TransformerQA(transformer_model, self.config)
        
        opt_config = self.config['train']['optim']['AdamW']
        for key, val in opt_config.items():
            mlflow.log_param(key, val)
        mlflow.log_param('nrof_classes', self.config['train']['num_classes'])
        #self.optimizer = Opt.SGD(self.model.parameters(), lr=.00001, momentum=0.9,
         #                        nesterov=True)
        #self.optimizer = Opt.RMSprop(self.model.parameters(), lr = .0001, alpha = 0.9)
        #'''
        self.optimizer = AdamW(self.model.parameters(),
                  lr = opt_config['lr'], 
                  eps = opt_config['eps'], 
                  weight_decay=opt_config['weight_decay'],
                )        
        #'''
        #self.total_steps = self.config['train']['nrof_steps']
        self.scheduler = get_cosine_schedule_with_warmup(self.optimizer, 
                                            num_warmup_steps = 50, 
                                            num_training_steps = self.config['train']['nrof_steps_for_shed'])
        self.crit = CrossEntropyLoss()
        self.model.to(self.config['train']['device'])
        #self.training_stats = []

        self.global_step = 0

        #mlflow.log_param('total_steps', self.total_steps)
    
    def save_model(self, step=None):
        torch.save({"model": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict(),
                    "scheduler": self.scheduler.state_dict(),
                    'global_step': self.global_step
                    },
                   os.path.join(self.config['paths']['ckpt_dir'], 
                                self.config['train']['exp_name'] + '_checkpoint' + ('_'+str(step) if step else '')))
        print('Model saved...')
    
    def load_model(self):
        if self.config['train']['use_intermediate_weights']:
            ckpt = torch.load(self.config['paths']['ckpt_to_load'],
                          map_location=self.config['train']['device'])
            model_st_dict = ckpt["model"]
            model_st_dict = dict([(key.replace('bert', 'transformer_model'), value) for key, value in model_st_dict.items()])
            self.model.load_state_dict(model_st_dict, strict=False)   
        else:
            ckpt = torch.load(self.config['paths']['ckpt_to_load'],
            #ckpt = torch.load(os.path.join(self.config['paths']['ckpt_dir'],
             #                           self.config['train']['exp_name'] + '_checkpoint'),
                            map_location=self.config['train']['device'])
            model_st_dict = ckpt["model"]
            self.model.load_state_dict(model_st_dict)   
            self.global_step = ckpt["global_step"] + 1
            self.optimizer.load_state_dict(ckpt["optimizer"])
            self.scheduler.load_state_dict(ckpt["scheduler"])
        print("Model loaded...")


    def train(self, 
              train_dataloader, 
              validation_dataloader,
              to_save=True):
        if self.config['train']['load_model']:
            self.load_model()
        self.model.train()
        print('GLOBAL STEP:', self.global_step)
        t0 = time()

        predicts, trues = [], []
        tr_losses = []
        cur_loss, nrof_steps, = 0., 0,
        nrof_cor_predicts_current, nrof_cor_predicts =0, 0
        nrof_samples_current, nrof_samples = 0, 0
        print('Train loader len:', len(train_dataloader))
        #class_correct = list(0. for i in range(self.config['train']['num_classes']))
        #class_total = list(0. for i in range(self.config['train']['num_classes']))
    
        for epoch in tqdm(range(self.config['train']['nrof_epochs'])):
            predicts, trues = [], []
            for step, batch in enumerate(train_dataloader):
                if epoch * len(train_dataloader) + step < self.global_step:
                    continue
                print('epoch: {} step: {}'.format(epoch, step))
                
                b_input_ids = batch[0].to(self.config['train']['device'])
                b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[2].to(self.config['train']['device'])       
                
                
                self.model.zero_grad()  
                try:
                    output = self.model(b_input_ids, 
                                        b_input_mask, b_labels)
                    #for param in self.model.parameters():
                     #   print(param)
                      #  break 
                        #param.requires_grad = False
                    loss = self.crit(output, b_labels)
                    cur_loss +=loss.item()
                    tr_losses.append(loss.item())
                    _, predicted = torch.max(output,-1)
                    predicts.extend(predicted.cpu().detach().numpy().flatten().tolist())
                    trues.extend(b_labels.cpu().detach().numpy().flatten().tolist())
                    #for i, label in enumerate(b_labels.cpu().detach().numpy()):
                     #   class_correct[int(label)] += (predicted == b_labels)[i].item()
                      #  class_total[int(label)] += 1

                    loss.backward()
                    #torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                    self.optimizer.step()
                    self.scheduler.step()
                    self.global_step+=1
                    nrof_steps+=1
                    nrof_samples+=len(b_labels)
                except RuntimeError as excp:
                    if "out of memory" in str(excp):
                        print("Out of memory!")
                        if hasattr(torch.cuda,'empty_cache'): 
                            torch.cuda.empty_cache() 
                    else:
                        raise excp
                
                if self.global_step % 1 == 0: 
                    elapsed = format_time(time() - t0) 
                    #if self.global_step % 10 == 0: 
                     #   self.save_model(step=self.global_step)
                      #  files.download(os.path.join('/content/ckpts', self.config['train']['exp_name'] + '_checkpoint' + '_'+str(self.global_step))) 

                    '''
                    if self.global_step % 20 == 0: 
                        try:
                            avg_val_loss, val_trues, val_predicts = self.validate(validation_dataloader)
                            avg_val_accuracy = balanced_accuracy_score(val_trues, val_predicts)
                            mlflow.log_metric("val_loss", avg_val_loss, step=self.global_step)
                            mlflow.log_metric("val_accuracy", avg_val_accuracy, step=self.global_step)
                            #mlflow.log_metric("val_f1_score", val_f1_score, step=self.global_step)
                            print('val loss: {}\nval bal accuracy: {}'.format(
                                avg_val_loss,  avg_val_accuracy))
                            plot_conf_matr(val_trues, val_predicts, title=str(self.global_step)+'_', 
                                           nrof_classes=self.config['train']['num_classes'], 
                                           classes_names=self.config['train']['classes_names'][self.config['train']['task_name']])
                            mlflow.log_artifact(str(self.global_step)+'_conf_matrix.png')
                            #self.model.train()
                            #torch.cuda.empty_cache()
                        except RuntimeError as excp:
                            if "out of memory" in str(excp):
                                print("Out of memory in VAL!")
                                if hasattr(torch.cuda,'empty_cache'): 
                                    torch.cuda.empty_cache() 
                            else:
                                raise excp
                    '''
                    if nrof_steps!=0:
                        tr_losses = tr_losses[-50:]
                        trues, predicts = trues[-50:], predicts[-50:] 
                        bal_tr_accuracy = balanced_accuracy_score(trues, predicts)
                        #tr_f1_score = f1_score(trues, predicts)
                        mlflow.log_metric("train_loss", cur_loss / nrof_steps, step=self.global_step) 
                        mlflow.log_metric("train_average_loss", np.mean(tr_losses),  step=self.global_step)
                        mlflow.log_metric("train_bal_accuracy", bal_tr_accuracy, step=self.global_step)
                        #mlflow.log_metric("train_f1_score", tr_f1_score, step=self.global_step)
                        if self.global_step % 5 == 0: 
                            print('Elapsed: {:}.'.format(elapsed))
                            print('tr loss: {}\ntr bal accuracy: {}'.format(cur_loss/nrof_steps, 
                                                                    bal_tr_accuracy))
                            print('trues: {}\npredicts:{}'.format(trues, predicts))
                        
                        cur_loss, nrof_steps = 0., 0
                        #trues, predicts = [], []
                    #val_acc,  val_classes_accs, val_loss =\
                    # self.validate(validation_dataloader)
                    
                    #print('val accs by classes:\n', '\n'.join(
                        #   [self.config['train']['class_to_heading_map'][i]+': '+str(round(x, 3)) for i, x in enumerate(val_classes_accs)]))
                    #print('val loss:', val_loss)
                    
                    #mlflow.log_metric("val_balanced_accuracy", np.mean(val_classes_accs))
                    
                    #plot_conf_matr(class_correct, class_total)
                    #mlflow.log_metric("train_loss", cur_loss / nrof_steps)
                    #tr_class_accs = np.asarray(class_correct) / np.asarray(class_total)
                    #tr_bal_acc = np.mean(tr_class_accs)
                    #tr_accuracy = balanced_accuracy_score(trues, predicts)
                    #tr_f1_score = f1_score(trues, predicts)
                    #mlflow.log_metric("train_accuracy", tr_accuracy)
                    #mlflow.log_metric("train_f1_score", tr_f1_score)
                    #mlflow.log_metric("train_accuracy_accum", float(nrof_cor_predicts)/nrof_samples)
                    
                    #mlflow.log_metric("train_balanced_accuracy", tr_bal_acc)
                    #print('tr loss: {}\ntr accuracy: {}'.format(cur_loss/nrof_steps, 
                     #                                           tr_accuracy))

                    
                    #predicts, trues = [], []

                if self.global_step % 20 == 0: # 5
                    self.save_model()
                    try:
                        copytree('/content/mlruns', self.config['paths']['mlruns'] + '_' + self.config['train']['exp_name'] +'_'+str(self.global_step))
                    except Exception as e:
                        print(e)
                        pass 
                if self.global_step % 100 == 0: # 5
                    self.save_model(step=self.global_step)

                training_time = (time() - t0)
            self.save_model(step=self.global_step)
        
        #self.save_model(step=self.global_step)
            try:
                copytree('/content/mlruns', self.config['paths']['mlruns'] + '_' + self.config['train']['exp_name'])
            except Exception as e:
                print(e)
                pass 
            


    def validate(self, validation_dataloader, to_load=False):
        if to_load:
            self.load_model()
        t1 = time()
        self.model.eval()
        probs, predicts, trues, imdb_ids = [], [], [], []
        nrof_steps, val_loss, nrof_cor_predicts, nrof_samples = 0, 0., 0, 0
        
        #class_correct = list(0. for i in range(self.config['train']['num_classes']))
        #class_total = list(0. for i in range(self.config['train']['num_classes']))

        for i, batch in tqdm(enumerate(validation_dataloader)):
            with torch.no_grad(): 
                b_input_ids = batch[0].to(self.config['train']['device'])
                b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[2].to(self.config['train']['device'])
                b_imdb_ids = batch[3].to(self.config['train']['device'])       

                output = self.model(b_input_ids, 
                                    b_input_mask, b_labels)
                val_loss += self.crit(output, b_labels).item()
                #_, predicted = torch.max(output,-1)
                sig_probs, predicted = torch.max(output.sigmoid(),-1)
                probs.extend(sig_probs.cpu().detach().numpy().flatten().tolist())
                predicts.extend(predicted.cpu().detach().numpy().flatten().tolist())
                trues.extend(b_labels.cpu().detach().numpy().flatten().tolist())
                imdb_ids.extend(b_imdb_ids.cpu().detach().numpy().flatten().tolist())
                #if_right = (predicted == b_labels).sum().item()
                #nrof_cor_predicts += if_right
                #for i, label in enumerate(b_labels.cpu().detach().numpy()):
                 #   class_correct[int(label)] += (predicted == b_labels)[i].item()
                  #  class_total[int(label)] += 1

                '''
                if_right = int(predicted == b_labels)
                nrof_cor_predicts += if_right
                #nrof_cor_predicts_current += if_right
                class_correct[b_labels.item()] += if_right
                class_total[b_labels.item()] += 1
                '''
                nrof_steps+=1
                nrof_samples+=len(b_labels)
                #nrof_samples_current+=len(b_labels)


                '''
                b_input_ids = batch[0].to(self.config['train']['device'])
                #b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[1].to(self.config['train']['device'])
                _,_, = self.model(b_input_ids, 
                            token_type_ids=None, 
                            #attention_mask=b_input_mask,
                            labels=b_labels,
                            return_dict=True)
                val_loss += outputs.loss.item()
                _, predicted = torch.max(outputs.logits,-1)
                c = (predicted == b_labels)
                nrof_cor_predicts += c.sum().item()
                for i in range(b_labels.size(0)):
                    label = b_labels[i]
                    if torch.is_tensor(label):
                        for j, ll in enumerate(label):
                            class_correct[ll] += c[i][j].squeeze().item()
                            class_total[ll] += 1
                    else:
                        class_correct[label] += c[i].squeeze().item()
                        class_total[label] += 1

                nrof_samples += len(b_labels)
                '''

                
        #avg_val_accuracy = balanced_accuracy_score(trues, predicts)
        #val_f1_score = f1_score(trues, predicts)
        
        avg_val_loss = val_loss / nrof_steps
        validation_time = (time() - t1)
        
        print("  Validation took: {:}".format(validation_time))
        
        return avg_val_loss, trues, predicts, imdb_ids, probs




In [ ]:
print(T.global_step)

In [ ]:
mlflow.end_run()

In [ ]:
T = Train(config)

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

tr_loader, val_loader, tst_loader = get_data_loaders()

In [ ]:
T = Train(config)

In [ ]:
avg_tst_loss, tst_trues, tst_predicts, tst_imdb_ids, tst_probs = T.validate(tst_loader, to_load=True)

In [ ]:
print(balanced_accuracy_score(tst_trues, tst_predicts))

In [ ]:
print(len(tst_trues))
print(tst_trues[0])
print(len(tst_imdb_ids))
print(tst_imdb_ids[0])
print(len(tst_probs))
print(tst_probs[0])

# Most confident errors

In [ ]:
with open('tst_trues_genre_acc_0_31.pickle', 'rb') as f:
    tst_trues = pickle.load(f)
with open('tst_predicts_genre_acc_0_31.pickle', 'rb') as f:
    tst_predicts = pickle.load(f)
with open('tst_imdb_ids_genre_acc_0_31.pickle', 'rb') as f:
    tst_imdb_ids = pickle.load(f)
with open('tst_probs_genre_acc_0_31.pickle', 'rb') as f:
    tst_probs = pickle.load(f)
    

In [ ]:
with open('tst_trues_awards_acc_0_74.pickle', 'rb') as f:
    tst_trues = pickle.load(f)
with open('tst_predicts_awards_acc_0_74.pickle', 'rb') as f:
    tst_predicts = pickle.load(f)
with open('tst_imdb_ids_awards_acc_0_74.pickle', 'rb') as f:
    tst_imdb_ids = pickle.load(f)
with open('tst_probs_awards_acc_0_74.pickle', 'rb') as f:
    tst_probs = pickle.load(f)

In [ ]:
print(tst_probs[:10])
print(tst_trues[:10])
print(tst_predicts[:10])
print(tst_imdb_ids[:10])

In [ ]:
sorted_inds = np.argsort(tst_probs)[::-1]
print(sorted_inds[:10])
print(np.asarray(tst_probs)[sorted_inds][:10])

In [ ]:
errors_mask = np.asarray(tst_trues)[sorted_inds] != np.asarray(tst_predicts)[sorted_inds]
print(errors_mask[:10])

In [ ]:
print(errors_mask[:50])

In [ ]:
sorted_inds = np.argsort(tst_probs)[::-1]
print(sorted_inds[:10])
print(np.asarray(tst_probs)[sorted_inds][:50])
print(np.asarray(tst_trues)[sorted_inds][:50])
print(np.asarray(tst_predicts)[sorted_inds][:50])
print(np.asarray(tst_imdb_ids)[sorted_inds][:50])

In [ ]:
print(np.asarray(tst_probs)[sorted_inds][errors_mask][:50])
print(np.asarray(tst_trues)[sorted_inds][errors_mask][:50])
print(np.asarray(tst_predicts)[sorted_inds][errors_mask][:50])
print(np.asarray(tst_imdb_ids)[sorted_inds][errors_mask][:50])

In [ ]:
with open('tst_trues_awards_acc_0_74.pickle', 'wb') as f:
    pickle.dump(tst_trues, f)
with open('tst_predicts_awards_acc_0_74.pickle', 'wb') as f:
    pickle.dump(tst_predicts, f)
with open('tst_imdb_ids_awards_acc_0_74.pickle', 'wb') as f:
    pickle.dump(tst_imdb_ids, f)
with open('tst_probs_awards_acc_0_74.pickle', 'wb') as f:
    pickle.dump(tst_probs, f)

In [ ]:
errors_df = pd.DataFrame({'movie':np.asarray(tst_imdb_ids)[sorted_inds][errors_mask].tolist(),
             'true_class':np.asarray(tst_trues)[sorted_inds][errors_mask].tolist(),
             'predicted_class':np.asarray(tst_predicts)[sorted_inds][errors_mask],
             'predicted_proba':np.asarray(tst_probs)[sorted_inds][errors_mask]})
errors_df.to_excel('genre_confident_errors.xlsx', index=False)

In [ ]:
errors_df = pd.DataFrame({'movie':np.asarray(tst_imdb_ids)[sorted_inds][errors_mask].tolist(),
             'true_class':np.asarray(tst_trues)[sorted_inds][errors_mask].tolist(),
             'predicted_class':np.asarray(tst_predicts)[sorted_inds][errors_mask],
             'predicted_proba':np.asarray(tst_probs)[sorted_inds][errors_mask]})
errors_df.to_excel('awards_confident_errors.xlsx', index=False)

In [ ]:
!cp -r '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/mlruns_script_awards_classification_10_scenes_average_with_nsp' .

In [ ]:
with mlflow.start_run(run_name=config['train']['exp_name'], run_id='cfbe31a258a846bfb7953f1d224aa530'):
    #'''
    avg_tst_loss, tst_trues, tst_predicts = T.validate(tst_loader, to_load=True)
    avg_tst_accuracy = balanced_accuracy_score(tst_trues, tst_predicts)
    tst_f1_score = f1_score(tst_trues, tst_predicts)
    plot_conf_matr(tst_trues, tst_predicts, 
                   file_path_prefix='test_steps_', title='Confusion matrix\nbalanced accuracy: {:.2f}\nf1 score: {:.2f}'.format(
                       avg_tst_accuracy, tst_f1_score))
    #'''
    mlflow.log_artifact('test_steps_conf_matrix.png')


In [ ]:
with mlflow.start_run(run_name=config['train']['exp_name'], run_id='cfbe31a258a846bfb7953f1d224aa530'):
    '''
    avg_tr_loss, tr_trues, tr_predicts = T.validate(tr_loader, to_load=True)
    avg_tr_accuracy = balanced_accuracy_score(tr_trues, tr_predicts)
    tr_f1_score = f1_score(tr_trues, tr_predicts)
    plot_conf_matr(tr_trues, tr_predicts, 
                   file_path_prefix='train_steps_', title='Confusion matrix\nbalanced accuracy: {:.2f}\nf1 score: {:.2f}'.format(
                       avg_tr_accuracy, tr_f1_score))
    '''
    mlflow.log_artifact('train_conf_matrix.png')

In [ ]:
copytree('/content/mlruns', config['paths']['mlruns'] + '_' + config['train']['exp_name'])

In [ ]:
shutil.rmtree('/content/mlruns')

In [ ]:
################### СКАЧАТЬ ДАННЫЕ!!!!!! ####################

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

with mlflow.start_run(run_name=config['train']['exp_name']):
    tr_loader, val_loader, tst_loader = get_data_loaders()
    T = Train(config)
    T.train(tr_loader, val_loader)

In [ ]:
mlflow.end_run()

In [ ]:
import shutil

In [ ]:
import shutil

shutil.rmtree('/content/mlruns')

In [ ]:
shutil.rmtree('/content/mlruns/0/52b570918d414f40910adc950ebeeb31')

In [ ]:
avg_val_loss, trues, predicts = T.validate(tst_loader)

In [ ]:
avg_tr_loss, tr_trues, tr_predicts = T.validate(tr_loader)

In [ ]:
avg_tr_accuracy = balanced_accuracy_score(tr_trues, tr_predicts)

In [ ]:
print(avg_tr_accuracy)

In [ ]:
avg_val_accuracy = balanced_accuracy_score(trues, predicts)

In [ ]:
print(avg_val_loss)
print(avg_val_accuracy)

In [ ]:
import mlflow
from pyngrok import ngrok

In [ ]:
!cp -r '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/mlruns_script_classification_with_val_765' .

In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
ngrok.kill()

# BoW model:

In [ ]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
imdb_ids_to_labels = {'tr': dict(zip(tr_imdb_ids, tr_labels)), 'val':dict(zip(val_imdb_ids, val_labels)), 'tst':dict(zip(tst_imdb_ids, tst_labels))}

In [ ]:
def get_train_test_split(tr_imdb_ids, val_imdb_ids, tst_imdb_ids, path_to_script_lemmas):
    processed_texts = []
    train_inds, val_inds, test_inds = [], [], []
    train_labels, val_labels, test_labels = [], [], []

    i = 0
    for file_name in tqdm(os.listdir(path_to_script_lemmas)):
        imdb_id = file_name.split('_')[1]
        with open(os.path.join(path_to_script_lemmas, file_name), 'r') as f:
            text = f.read()
        processed_texts.append(text)
        if imdb_id in tr_imdb_ids:
            train_inds.append(i)
            train_labels.append(imdb_ids_to_labels['tr'][imdb_id].item())
        elif imdb_id in val_imdb_ids:
            val_inds.append(i)
            val_labels.append(imdb_ids_to_labels['val'][imdb_id].item())
        elif imdb_id in tst_imdb_ids:
            test_inds.append(i)
            test_labels.append(imdb_ids_to_labels['tst'][imdb_id].item())
        i+=1
    return processed_texts, train_inds, val_inds, test_inds, train_labels, val_labels, test_labels

In [ ]:
processed_texts, train_inds, val_inds, test_inds, train_labels, val_labels, test_labels = get_train_test_split(tr_imdb_ids, val_imdb_ids, tst_imdb_ids, config['paths']['script_lemmas'])

In [ ]:
print(train_inds[:20])
print(train_labels[:20])

In [ ]:
results_df = {'max_features':[], 'binary':[],'norm':[],'use_idf':[], 'smooth_idf':[],'sublinear_tf':[], 'Train balanced accuracy': [], 'Test balanced accuracy':[], 'Test f1-score':[]}
#y = DP.labels # old dataset!!
#y = processed_labels
nrof_exps = 0
for max_features in (1500, 2000):
    for binary in (False, True):
        for norm in ('l1', 'l2'):
            for sublinear_tf in (False, True):
                for use_idf in (False, True):
                    if use_idf:
                        for smooth_idf in (False, True):
                            tfidfconverter = TfidfVectorizer(max_features=max_features, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=binary,norm=norm,use_idf=use_idf,smooth_idf=smooth_idf,sublinear_tf=sublinear_tf)
                            X = tfidfconverter.fit_transform(processed_texts).toarray()
                            X_train = X[train_inds]
                            X_val = X[val_inds]
                            X_test = X[test_inds]
                            y_train = train_labels
                            y_val = val_labels
                            y_test = test_labels
                            #X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
                            #X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=11)
                            clf = svm.LinearSVC(class_weight='balanced', random_state=11)
                            clf.fit(X_train, y_train)
                            #clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
                            y_pred = clf.predict(X_test)
                            tr_y_pred = clf.predict(X_train)
                            bal_acc = balanced_accuracy_score(y_test, y_pred)
                            tr_bal_acc = balanced_accuracy_score(y_train,tr_y_pred)
                            #f1score = f1_score(y_test,y_pred)
                            #print(balanced_accuracy_score(y_test,y_pred))
                            #print(f1_score(y_test,y_pred))
                            results_df['max_features'].append(max_features)
                            results_df['binary'].append(binary)
                            results_df['norm'].append(norm)
                            results_df['use_idf'].append(use_idf)
                            results_df['smooth_idf'].append(use_idf)
                            results_df['sublinear_tf'].append(sublinear_tf)
                            results_df['Train balanced accuracy'].append(round(tr_bal_acc, 4))
                            results_df['Test balanced accuracy'].append(round(bal_acc, 4))
                            #results_df['Test f1-score'].append(round(f1score, 4))
                            results_df['Test f1-score'].append(0)
                            #plot_conf_matr(y_test,y_pred, classes_names=['minor', 'main'], title=)
                            #results_df['SVM linear results'].append('Train balanced accuracy: {:.4f}\nTest balanced accuracy: {:.4f}\nTest f1-score: {:.4f}'.format(tr_bal_acc, bal_acc, f1score))
                            nrof_exps+=1
                            print('nrof exps', nrof_exps)
                    else:
                        tfidfconverter = TfidfVectorizer(max_features=max_features, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=binary,norm=norm,use_idf=use_idf,sublinear_tf=sublinear_tf)
                        X = tfidfconverter.fit_transform(processed_texts).toarray()
                        X_train = X[train_inds]
                        X_val = X[val_inds]
                        X_test = X[test_inds]
                        y_train = train_labels
                        y_val = val_labels
                        y_test = test_labels
                        clf = svm.LinearSVC(class_weight='balanced', random_state=11)
                        clf.fit(X_train, y_train)
                        #clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
                        y_pred = clf.predict(X_test)
                        tr_y_pred = clf.predict(X_train)
                        bal_acc = balanced_accuracy_score(y_test,y_pred)
                        tr_bal_acc = balanced_accuracy_score(y_train,tr_y_pred)
                        #f1score = f1_score(y_test,y_pred)
                        #print(balanced_accuracy_score(y_test,y_pred))
                        #print(f1_score(y_test,y_pred))
                        results_df['max_features'].append(max_features)
                        results_df['binary'].append(binary)
                        results_df['norm'].append(norm)
                        results_df['use_idf'].append(use_idf)
                        results_df['smooth_idf'].append(False)
                        results_df['sublinear_tf'].append(sublinear_tf)
                        results_df['Train balanced accuracy'].append(round(tr_bal_acc, 4))
                        results_df['Test balanced accuracy'].append(round(bal_acc, 4))
                        #results_df['Test f1-score'].append(round(f1score, 4))
                        results_df['Test f1-score'].append(0)
                        #results_df['SVM linear results'].append('Train balanced accuracy: {:.4f}\nTest balanced accuracy: {:.4f}\nTest f1-score: {:.4f}'.format(tr_bal_acc, bal_acc, f1score))
                        nrof_exps+=1
                        print('nrof exps', nrof_exps)

In [ ]:
results_df = pd.DataFrame(results_df)

In [ ]:
results_df

In [ ]:
results_df.to_excel('BoW_validation_results_script_awards_new.xlsx', index=False)

In [ ]:
results_df.sort_values(by=['Test f1-score'], ascending=False)

In [ ]:
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=False,norm='l2',use_idf=False,smooth_idf=False,sublinear_tf=True)
X = tfidfconverter.fit_transform(processed_texts).toarray()
X_train = X[train_inds]
X_val = X[val_inds]
X_test = X[test_inds]
y_train = train_labels
y_val = val_labels
y_test = test_labels
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=11)
clf = svm.LinearSVC(class_weight='balanced', random_state=11)
clf.fit(X_train, y_train)
#clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
y_pred = clf.predict(X_test)
tr_y_pred = clf.predict(X_train)
bal_acc = balanced_accuracy_score(y_test, y_pred)
tr_bal_acc = balanced_accuracy_score(y_train,tr_y_pred)
#f1score = f1_score(y_test,y_pred)

In [ ]:
print(len(y_test))

In [ ]:
print(y_test[:20])

In [ ]:
print(len(y_pred))

In [ ]:
print(y_pred[:20])

In [ ]:
plot_conf_matr(y_test, y_pred, title='Confusion matrix\nBalanced accuracy: {}'.format(bal_acc), file_path_prefix='test_',
               nrof_classes = config['train']['num_classes'],  classes_names=config['train']['classes_names'][config['train']['task_name']])